In [1]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [2]:
#lấy data danh sách report
df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vSuXl7AnpSA5j_KrgFO2zhLn8DsD20RVuuG7g6Y7F9y5EbiPHT05ug_m3eh8MBOaMDvNmk-DX4g0igF/pub?gid=1043526978&single=true&output=csv")
cleancols(df)
df.columns =lower_col(df)
df_aut = pd.melt(df,id_vars=['tenreport', 'linkreport', 'type','id', 'vw'])
df_aut=df_aut[~df_aut['value'].isna()]
df_aut.rename(columns={'value':'accessgroup'},inplace=True)

#lấy data phân quyền detail tách theo phân quyền CA & non CA
df_group = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vSuXl7AnpSA5j_KrgFO2zhLn8DsD20RVuuG7g6Y7F9y5EbiPHT05ug_m3eh8MBOaMDvNmk-DX4g0igF/pub?gid=2075224509&single=true&output=csv")
cleancols(df_group)
df_group.columns=lower_col(df_group)    
dk2= df_group['csv_link']!='CA'
df_link = df_group[~df_group['csv_link'].isna()]
link=df_link['csv_link'][df_link['accessgroupcongtyphongbanlevel']!='CA']
ca_link=df_link['csv_link'][df_link['accessgroupcongtyphongbanlevel']=='CA']

# #Join phân quyền theo Access Group
df0 = pd.DataFrame()
for i in link:
    df = pd.read_csv(i)
    df0 = pd.concat([df0,df])
dropdup(df0,3)
df0['accessgroup'] = df0['congty']+'-'+df0['phongban']+'-'+df0['chucdanh']
full_df = pd.merge(df_aut,df0,how='inner', on='accessgroup')

# #Join phân quyền theo chuc danh đối với cấp CA
df1 = pd.DataFrame()
for i in ca_link:
    df = pd.read_csv(i)
    df1 = pd.concat([df1,df])
full_df_ca = pd.merge(df_aut,df1,how='inner',left_on='accessgroup', right_on='chucdanh')

#nối data & inset to BQ
full_df = pd.concat([full_df,full_df_ca])
df_mail = full_df[['email','tencvbh']]
df_mail['email1'] = df_mail['email'].str.lower()
df_mail = dropdup(df_mail,3,subset='email1')
df_mail['tencvbh1'] =df_mail['tencvbh']
df_mail=df_mail[['email','tencvbh1','email1']]
full_df = pd.merge(full_df,df_mail,how='left',on='email')
full_df.drop(['email','tencvbh'], axis=1,inplace=True)
full_df.rename(columns={'email1':'email',
                            'tencvbh1':'tencvbh'
                            }, inplace=True)


full_df['inserted_at']=datetime.now()

C:\Users\Van Quang Duy\AppData\Local\Temp\ipykernel_16568\2675974430.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mail['email1'] = df_mail['email'].str.lower()


In [8]:
# full_df.to_clipboard()

In [3]:
full_df = dropdup(full_df, 1, subset=['id', 'manv'])

In [4]:
bq_values_insert(full_df, "d_phanquyen_tonghop_sep",3)

In [ ]:
# SELECT tenreport, id, manv, type, vw FROM `spatial-vision-343005.biteam.d_phanquyen_tonghop_sep`

In [1]:
upload_file_to_bucket_with_metadata(blobname="public/duy.csv", file="report_users.csv")

In [2]:
client=storage.Client()

In [22]:
bucketname='django_media_biteam'
blobname="public/test1.json"

In [23]:
bucket = client.get_bucket(bucketname)
blob = bucket.blob(blobname)

In [24]:
bucket = client.get_bucket('django_media_biteam')

In [25]:
blob = bucket.blob(blobname)

In [26]:
blob.cache_control = 'no-cache'

In [27]:
metadata = {'Cache-Control': 'no-cache'}
blob.metadata = metadata

In [29]:
blob.upload_from_filename("user_reports1.json")